# Introduction to Happywhale competition

To have a first look at the [data](https://www.kaggle.com/c/happy-whale-and-dolphin/data), you have to add the data through the Kaggle UI. They are afterwards located in */kaggle/input/happy-whale-and-dolphin* folder.

In [8]:
!ls -l /kaggle/input/happy-whale-and-dolphin

total 4668
-rw-r--r-- 1 nobody nogroup 2404234 Feb  1 16:49 sample_submission.csv
drwxr-xr-x 2 nobody nogroup       0 Feb  1 16:51 test_images
-rw-r--r-- 1 nobody nogroup 2371769 Feb  1 16:51 train.csv
drwxr-xr-x 2 nobody nogroup       0 Feb  1 16:57 train_images


Let’s load the metadata in 'train.csv' file. 

In [13]:
import pandas as pd
df = pd.read_csv("/kaggle/input/happy-whale-and-dolphin/train.csv")
df

,image,species,individual_id
0,00021adfb725ed.jpg,melon_headed_whale,cadddb1636b9
1,000562241d384d.jpg,humpback_whale,1a71fbb72250
2,0007c33415ce37.jpg,false_killer_whale,60008f293a2b
3,0007d9bca26a99.jpg,bottlenose_dolphin,4b00fe572063
4,00087baf5cef7a.jpg,humpback_whale,8e5253662392
...,...,...,...
51028,fff639a7a78b3f.jpg,beluga,5ac053677ed1
51029,fff8b32daff17e.jpg,cuviers_beaked_whale,1184686361b3
51030,fff94675cc1aef.jpg,blue_whale,5401612696b9
51031,fffbc5dd642d8c.jpg,beluga,4000b3d7c24e


We can notice that our train data describes 51 033 images with one ID field for the photography filename and 2 others fields. The latter describe the animal specy and which individual it is.

Whales are a species included in the *Cetacea* infraorder.
Dolphin are a species included in the *Cetacea* infraorder and the *Delphinidae* superfamily.
Beluga, or more commonly called **white whale**, is a specy from the *Cetacea* infraorder and the *Delphinidae* superfamily.
Globis, or more commonly called **pilot whale**, is a specy from the *Cetacea* infraorder and the *Delphinidae* superfamily.

In [40]:
species = df.drop(columns = ['individual_id']).groupby(['species']).count().rename(columns = { 'image': 'counts'})
individuals = df.drop(columns = ['species']).groupby(['individual_id']).count().rename(columns = { 'image': 'counts'})
species
individuals

,counts
individual_id,
0013f1f5f2f0,1
001618e0a31e,1
0018a0f40586,1
00191b9a2f6f,1
001a09e48b2d,4
...,...
fff35b58c591,1
fff591b41cce,4
fffb11ff4575,4
